In [98]:
# Code to import the .xlsx file.

import pandas as pd
import sys
reload(sys)
sys.setdefaultencoding('utf8')

# Isabelle: change the name below to the name of the excel file you're importing. Make sure it's in the same directory
df = pd.read_excel('FactBase_0417_ca.xlsx', index_col=0)
df.fillna(-1, inplace=True)
df = df.apply(lambda x: x.astype(str).str.lower())
df.replace("additional accounting", "additional attribute", inplace=True)
df.replace("additional reporting", "additional attribute", inplace=True)

df

In [99]:
# This is a check to see the number of unique PR keys in the file. Should be 171, unless we add / remove from iteration 3
df1 = df
len(df1.loc[ : , 'PRKey' ].unique())

33

In [100]:
# Isbelle: This is the name of the output file. Change below to what you want the output file to be, you run this block TWICE.
# Once at the beginning, and then again at the end.
writer = pd.ExcelWriter('Summary0417_v1.xlsx', engine='xlsxwriter')

In [101]:
# This block removes all of the reference data feeds, since it 3 was only analysis on transaction / event feeds

df = df[~df.loc[ : , 'PRKey' ].str.contains("ref")]
df = df[~df.loc[ : , 'PRKey' ].str.contains(".prn")]
df = df[~df.loc[ : , 'PRKey' ].str.contains("gf.txt - chess service billing")]
df = df[~df.loc[ : , 'PRKey' ].str.contains("data mart")]
df = df[~df.loc[ : , 'PRKey' ].str.contains("gf.txt - pheonix")]
df = df[~df.loc[ : , 'PRKey' ].str.contains("gf.txt - residence distribution")]
df = df[~df.loc[ : , 'PRKey' ].str.contains("gf.txt - bif, s&c")]

# This is a check to see the count of unique PR Keys after we remove all the reference data. It should be 133, unless we have
# added or removed feeds

len(df.loc[ : , 'PRKey' ].unique())

22

In [102]:
# This sums the count of feeds using a particular strategy

summary = pd.DataFrame({'Feed Count': df.groupby(['Reqcategory','FinancialReportingRequirements', 'BUProposedStrategytoMeetRequirements'])['BUProposedStrategytoMeetRequirements'].count()}).reset_index()
summary_file = pd.DataFrame({'Feed Count': df.groupby(['Reqcategory','Typename','FinancialReportingRequirements', 'BUProposedStrategytoMeetRequirements'])['BUProposedStrategytoMeetRequirements'].count()}).reset_index()

In [103]:
# This creates the table with all the files using a particular file type, by BG and BU

file_types_all = pd.DataFrame(df.groupby(['Typename', 'BG', 'BU','PRKey'])['Typename'].count())
file_types_all

Typename
Typename       BG     BU             PRKey                            
ais            Canada individual     if.txt                         65
                                     l2.txt                         65
                                     lu.txt                         68
                                     m1.txt                         63
                                     m3.txt                         63
                                     py.txt                         64
                                     pz.txt                         67
                                     sj.txt                         68
direct         Canada individual     cf.txt                         71
                                     f3.txt                         71
                                     ml.txt                         71
lawson         Canada individual     l3.txt                         69
new gb aso tbf Canada group benefits gf.txt - aso accounting        25
new gb ebf     Canada group benefits g3.txt - disability            61
                                     g3.txt - medical dental        55
                                     gf.txt                         73
new grs ebf    Canada grs            yx.txt                        140
pre-direct     Canada individual     ad.txt                        113
                                     ju.txt                        113
                                     lf.txt                        113
                                     md.txt                        111
pseudo-event   Canada grs            yl.txt                         31

In [104]:
# This creates the table with the coutn and percentage of all IFRS 17 attributes that are not "undecided"

summary_derived = df[(df['Reqcategory']=='ifrs 17') & (df['BUProposedStrategytoMeetRequirements']!='undecided')]
bu_strategy = pd.DataFrame({'# Attrb Decided': summary_derived.groupby(['BG', 'BU','PRKey'])['PRKey'].count()}).reset_index()
bu_strategy['Percentage Decided'] = bu_strategy['# Attrb Decided'] / 17 * 100
bu_strategy


,BG,BU,PRKey,# Attrb Decided,Percentage Decided
0,Canada,group benefits,g3.txt - disability,15,88.235294
1,Canada,group benefits,g3.txt - medical dental,15,88.235294
2,Canada,group benefits,gf.txt,8,47.058824
3,Canada,grs,yx.txt,15,88.235294
4,Canada,individual,ad.txt,11,64.705882
5,Canada,individual,cf.txt,11,64.705882
6,Canada,individual,f3.txt,11,64.705882
7,Canada,individual,if.txt,14,82.352941
8,Canada,individual,ju.txt,11,64.705882
9,Canada,individual,l2.txt,13,76.470588


In [105]:
file_types = df.Typename.unique()

file_types

array(['new gb ebf', 'new gb aso tbf', 'new grs ebf', 'pseudo-event',
       'ais', 'lawson', 'direct', 'pre-direct'], dtype=object)

In [106]:
# File Type: Attribute count total
attributes_file = df.FinancialReportingRequirements.unique()
attributes_file_count = {}
for j in file_types:
    nested = {}
    for i in attributes_file:
        name = df.loc[(df['Typename'].isin([j])) & (df['FinancialReportingRequirements'].isin([i]))]
        nested[i] = name.FinancialReportingRequirements.count()
    attributes_file_count[j] = nested

attributes_file_count

{'ais': {'1st billing pause': 0,
  '2nd billing pause': 0,
  'abc account number': 8,
  'account currency code': 0,
  'account number': 8,
  'account: claims (e.g. death, annuity, etc.)': 8,
  'account: directly attributable acquisition costs': 8,
  'account: investment component (how you identify the investment component of a claim/benefit transaction)': 8,
  'account: premiums (first year, renewal, single)': 8,
  'accounting - num of det line rec': 0,
  'accounting - num of transactions': 0,
  'accounting - total credit amount': 0,
  'accounting - total debit amount': 0,
  'accounting bases of feeds (i.e. ifrs 4, ifrs 17, usgaap, etc.)': 8,
  'accounting run date': 0,
  'accounting unit': 0,
  'address 1 (optional)': 0,
  'address 2 (optional)': 0,
  'adjustment amount': 0,
  'admin type': 0,
  'advisor id': 0,
  'agency or department ident': 0,
  'ais - appfill': 8,
  'ais - descrfl': 8,
  'ais - match key': 8,
  'ais - refno': 8,
  'ais - rescent': 8,
  'ais - type code': 8,
  'all

In [107]:
# File Type: Total Attribute count
summary_file['Percentage'] = ""
for file_type in attributes_file_count:
    for key in attributes_file_count[file_type]:
        new = summary_file.loc[(summary_file['Typename'].isin([file_type])) & (summary_file['FinancialReportingRequirements'].isin([key]))]
        new['Percentage'] = new['Feed Count'] / attributes_file_count[file_type][key] * 100
        summary_file.update(new)

summary_file

,Reqcategory,Typename,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Feed Count,Percentage
0,additional attribute,ais,abc account number,attribute in feed,8.0,100
1,additional attribute,ais,ais - appfill,attribute in feed,8.0,100
2,additional attribute,ais,ais - descrfl,attribute in feed,8.0,100
3,additional attribute,ais,ais - match key,attribute in feed,8.0,100
4,additional attribute,ais,ais - refno,attribute in feed,8.0,100
5,additional attribute,ais,ais - rescent,attribute in feed,8.0,100
6,additional attribute,ais,ais - type code,attribute in feed,8.0,100
7,additional attribute,ais,association of nonpar riders with the base plan,derived from feed attributes,2.0,100
8,additional attribute,ais,base account,attribute in feed,8.0,100
9,additional attribute,ais,branch,attribute in feed,8.0,100


In [108]:
summary_file['Strat Percent Max'] = summary_file.groupby(['Typename','FinancialReportingRequirements'])['Percentage'].transform(max)
summary_file

,Reqcategory,Typename,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Feed Count,Percentage,Strat Percent Max
0,additional attribute,ais,abc account number,attribute in feed,8.0,100,100.0
1,additional attribute,ais,ais - appfill,attribute in feed,8.0,100,100.0
2,additional attribute,ais,ais - descrfl,attribute in feed,8.0,100,100.0
3,additional attribute,ais,ais - match key,attribute in feed,8.0,100,100.0
4,additional attribute,ais,ais - refno,attribute in feed,8.0,100,100.0
5,additional attribute,ais,ais - rescent,attribute in feed,8.0,100,100.0
6,additional attribute,ais,ais - type code,attribute in feed,8.0,100,100.0
7,additional attribute,ais,association of nonpar riders with the base plan,derived from feed attributes,2.0,100,100.0
8,additional attribute,ais,base account,attribute in feed,8.0,100,100.0
9,additional attribute,ais,branch,attribute in feed,8.0,100,100.0


In [109]:
attributes = df.FinancialReportingRequirements.unique()
attributes
attributes_count = {}
for i in attributes:
    name = df.loc[df['FinancialReportingRequirements'].isin([i])]
    attributes_count[i] = name.FinancialReportingRequirements.count()

attributes_count

{'1st billing pause': 1,
 '2nd billing pause': 1,
 'abc account number': 8,
 'account currency code': 1,
 'account number': 22,
 'account: claims (e.g. death, annuity, etc.)': 21,
 'account: directly attributable acquisition costs': 21,
 'account: investment component (how you identify the investment component of a claim/benefit transaction)': 21,
 'account: premiums (first year, renewal, single)': 21,
 'accounting - num of det line rec': 4,
 'accounting - num of transactions': 4,
 'accounting - total credit amount': 4,
 'accounting - total debit amount': 4,
 'accounting bases of feeds (i.e. ifrs 4, ifrs 17, usgaap, etc.)': 22,
 'accounting run date': 8,
 'accounting unit': 1,
 'address 1 (optional)': 4,
 'address 2 (optional)': 4,
 'adjustment amount': 1,
 'admin type': 1,
 'advisor id': 4,
 'agency or department ident': 4,
 'ais - appfill': 8,
 'ais - descrfl': 8,
 'ais - match key': 8,
 'ais - refno': 8,
 'ais - rescent': 8,
 'ais - type code': 8,
 'allocation': 1,
 'amount': 22,
 '

In [110]:
summary['Percentage'] = ""
for key in attributes_count:
    new = summary.loc[(summary['FinancialReportingRequirements'].isin([key]))]
    new['Percentage'] = new['Feed Count'] / attributes_count[key] * 100
    summary.update(new)

summary

,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Feed Count,Percentage
0,additional attribute,1st billing pause,attribute in feed,1.0,100
1,additional attribute,2nd billing pause,attribute in feed,1.0,100
2,additional attribute,abc account number,attribute in feed,8.0,100
3,additional attribute,account currency code,attribute in feed,1.0,100
4,additional attribute,accounting - num of det line rec,attribute in feed,4.0,100
5,additional attribute,accounting - num of transactions,attribute in feed,4.0,100
6,additional attribute,accounting - total credit amount,attribute in feed,4.0,100
7,additional attribute,accounting - total debit amount,attribute in feed,4.0,100
8,additional attribute,accounting run date,attribute in feed,8.0,100
9,additional attribute,accounting unit,attribute in feed,1.0,100


In [111]:
summary['Strat Percent Max'] = summary.groupby(['FinancialReportingRequirements'])['Percentage'].transform(max)
summary

,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Feed Count,Percentage,Strat Percent Max
0,additional attribute,1st billing pause,attribute in feed,1.0,100,100.000000
1,additional attribute,2nd billing pause,attribute in feed,1.0,100,100.000000
2,additional attribute,abc account number,attribute in feed,8.0,100,100.000000
3,additional attribute,account currency code,attribute in feed,1.0,100,100.000000
4,additional attribute,accounting - num of det line rec,attribute in feed,4.0,100,100.000000
5,additional attribute,accounting - num of transactions,attribute in feed,4.0,100,100.000000
6,additional attribute,accounting - total credit amount,attribute in feed,4.0,100,100.000000
7,additional attribute,accounting - total debit amount,attribute in feed,4.0,100,100.000000
8,additional attribute,accounting run date,attribute in feed,8.0,100,100.000000
9,additional attribute,accounting unit,attribute in feed,1.0,100,100.000000


In [112]:
# File Type: Derived Attribute Analysis

is_derived_file =  df['BUProposedStrategytoMeetRequirements']=='derived from feed attributes'
summary_derived_file = df[is_derived_file]

total_derived_file = summary_derived_file.groupby(['Typename','FinancialReportingRequirements', 'BUProposedStrategytoMeetRequirements']).size().reset_index(name='Total_Derived')
total_derived_file = total_derived_file[['Typename','FinancialReportingRequirements', 'Total_Derived']]
total_derived_file
duplicates_file = summary_derived_file.groupby(['Typename','Reqcategory', 'FinancialReportingRequirements', "BUProposedStrategytoMeetRequirements",'Attribute1_AttributeDefinition_ifnetnewdataattribute','Attribute2_AttributeDefinition_ifnetnewdataattribute','Attribute3_AttributeDefinition_ifnetnewdataattribute','Attribute4_AttributeDefinition_ifnetnewdataattribute','Attribute5_AttributeDefinition_ifnetnewdataattribute','Attribute6_AttributeDefinition_ifnetnewdataattribute','Attribute7_AttributeDefinition_ifnetnewdataattribute','Attribute8_AttributeDefinition_ifnetnewdataattribute','Attribute9_AttributeDefinition_ifnetnewdataattribute','Attribute10_AttributeDefinition_ifnetnewdataattribute']).size().reset_index(name='Count')
df_merge_col_file = pd.merge(duplicates_file, total_derived_file, on=['Typename','FinancialReportingRequirements'])
df_merge_col_file['Percentage'] = df_merge_col_file['Count'] / df_merge_col_file['Total_Derived'] * 100
df_merge_col_file


,Typename,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Attribute1_AttributeDefinition_ifnetnewdataattribute,Attribute2_AttributeDefinition_ifnetnewdataattribute,Attribute3_AttributeDefinition_ifnetnewdataattribute,Attribute4_AttributeDefinition_ifnetnewdataattribute,Attribute5_AttributeDefinition_ifnetnewdataattribute,Attribute6_AttributeDefinition_ifnetnewdataattribute,Attribute7_AttributeDefinition_ifnetnewdataattribute,Attribute8_AttributeDefinition_ifnetnewdataattribute,Attribute9_AttributeDefinition_ifnetnewdataattribute,Attribute10_AttributeDefinition_ifnetnewdataattribute,Count,Total_Derived,Percentage
0,ais,additional attribute,association of nonpar riders with the base plan,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000
1,ais,additional attribute,minor product,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000
2,ais,additional attribute,natural account,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000
3,ais,gl accounting requirements,account number,derived from feed attributes,base account,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000
4,ais,gl accounting requirements,debit / credit,derived from feed attributes,amount,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000
5,ais,gl accounting requirements,legal entity,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000
6,ais,gl accounting requirements,major product (segment/lob),derived from feed attributes,line of business,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000
7,ais,ifrs 17,"account: claims (e.g. death, annuity, etc.)",derived from feed attributes,claims,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000
8,ais,ifrs 17,"account: premiums (first year, renewal, single)",derived from feed attributes,account,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000
9,ais,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,pr key,line of business,-1,-1,-1,-1,-1,-1,-1,-1,2,3,66.666667


In [113]:
# File Type: Derived Max
df_merge_col_file['Derived Percent Max'] = df_merge_col_file.groupby(['Typename','FinancialReportingRequirements'])['Percentage'].transform(max)
df_merge_col_file

# df_merge_col

,Typename,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Attribute1_AttributeDefinition_ifnetnewdataattribute,Attribute2_AttributeDefinition_ifnetnewdataattribute,Attribute3_AttributeDefinition_ifnetnewdataattribute,Attribute4_AttributeDefinition_ifnetnewdataattribute,Attribute5_AttributeDefinition_ifnetnewdataattribute,Attribute6_AttributeDefinition_ifnetnewdataattribute,Attribute7_AttributeDefinition_ifnetnewdataattribute,Attribute8_AttributeDefinition_ifnetnewdataattribute,Attribute9_AttributeDefinition_ifnetnewdataattribute,Attribute10_AttributeDefinition_ifnetnewdataattribute,Count,Total_Derived,Percentage,Derived Percent Max
0,ais,additional attribute,association of nonpar riders with the base plan,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000,100.000000
1,ais,additional attribute,minor product,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000,100.000000
2,ais,additional attribute,natural account,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000,100.000000
3,ais,gl accounting requirements,account number,derived from feed attributes,base account,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000,100.000000
4,ais,gl accounting requirements,debit / credit,derived from feed attributes,amount,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000,100.000000
5,ais,gl accounting requirements,legal entity,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000,100.000000
6,ais,gl accounting requirements,major product (segment/lob),derived from feed attributes,line of business,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000,100.000000
7,ais,ifrs 17,"account: claims (e.g. death, annuity, etc.)",derived from feed attributes,claims,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000,100.000000
8,ais,ifrs 17,"account: premiums (first year, renewal, single)",derived from feed attributes,account,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000,100.000000
9,ais,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,pr key,line of business,-1,-1,-1,-1,-1,-1,-1,-1,2,3,66.666667,66.666667


In [114]:
# Derived Attribute Analysis

is_derived =  df['BUProposedStrategytoMeetRequirements']=='derived from feed attributes'
summary_derived = df[is_derived]
summary_derived
total_derived = summary_derived.groupby(['FinancialReportingRequirements', 'BUProposedStrategytoMeetRequirements']).size().reset_index(name='Total_Derived')
total_derived = total_derived[['FinancialReportingRequirements', 'Total_Derived']]
total_derived
duplicates = summary_derived.groupby(['Reqcategory', 'FinancialReportingRequirements', "BUProposedStrategytoMeetRequirements",'Attribute1_AttributeDefinition_ifnetnewdataattribute','Attribute2_AttributeDefinition_ifnetnewdataattribute','Attribute3_AttributeDefinition_ifnetnewdataattribute','Attribute4_AttributeDefinition_ifnetnewdataattribute','Attribute5_AttributeDefinition_ifnetnewdataattribute','Attribute6_AttributeDefinition_ifnetnewdataattribute','Attribute7_AttributeDefinition_ifnetnewdataattribute','Attribute8_AttributeDefinition_ifnetnewdataattribute','Attribute9_AttributeDefinition_ifnetnewdataattribute','Attribute10_AttributeDefinition_ifnetnewdataattribute']).size().reset_index(name='Count')
df_merge_col = pd.merge(duplicates, total_derived, on=['FinancialReportingRequirements'])
df_merge_col
df_merge_col['Percentage'] = df_merge_col['Count'] / df_merge_col['Total_Derived'] * 100
df_merge_col

,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Attribute1_AttributeDefinition_ifnetnewdataattribute,Attribute2_AttributeDefinition_ifnetnewdataattribute,Attribute3_AttributeDefinition_ifnetnewdataattribute,Attribute4_AttributeDefinition_ifnetnewdataattribute,Attribute5_AttributeDefinition_ifnetnewdataattribute,Attribute6_AttributeDefinition_ifnetnewdataattribute,Attribute7_AttributeDefinition_ifnetnewdataattribute,Attribute8_AttributeDefinition_ifnetnewdataattribute,Attribute9_AttributeDefinition_ifnetnewdataattribute,Attribute10_AttributeDefinition_ifnetnewdataattribute,Count,Total_Derived,Percentage
0,additional attribute,association of nonpar riders with the base plan,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,6,6,100.000000
1,additional attribute,corporation,derived from feed attributes,issue,line of business,transaction type,account number,-1,-1,-1,-1,-1,-1,1,1,100.000000
2,additional attribute,deduction amount,derived from feed attributes,deduction amount,deduction code,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000
3,additional attribute,minor product,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,12,66.666667
4,additional attribute,minor product,derived from feed attributes,minor product,-1,-1,-1,-1,-1,-1,-1,-1,-1,4,12,33.333333
5,additional attribute,natural account,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,6,6,100.000000
6,additional attribute,segment,derived from feed attributes,issue,line of business,transaction type,account number,case reimbursement code,-1,-1,-1,-1,-1,1,1,100.000000
7,banking information,bank disbursement date,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,4,4,100.000000
8,banking information,bank message,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,4,4,100.000000
9,gl accounting requirements,account number,derived from feed attributes,amount,pdd indicator,ooc claims indicator / diff currency indicator,ooc claim amount / combined cheque amount,ooc currency code of payment by provider (jack...,-1,-1,-1,-1,-1,1,14,7.142857


In [115]:
attribute_list_ifrs17= df_merge_col[ df_merge_col['Reqcategory']=="ifrs 17"]
attribute_list_ifrs17

,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Attribute1_AttributeDefinition_ifnetnewdataattribute,Attribute2_AttributeDefinition_ifnetnewdataattribute,Attribute3_AttributeDefinition_ifnetnewdataattribute,Attribute4_AttributeDefinition_ifnetnewdataattribute,Attribute5_AttributeDefinition_ifnetnewdataattribute,Attribute6_AttributeDefinition_ifnetnewdataattribute,Attribute7_AttributeDefinition_ifnetnewdataattribute,Attribute8_AttributeDefinition_ifnetnewdataattribute,Attribute9_AttributeDefinition_ifnetnewdataattribute,Attribute10_AttributeDefinition_ifnetnewdataattribute,Count,Total_Derived,Percentage
43,ifrs 17,"account: claims (e.g. death, annuity, etc.)",derived from feed attributes,claims,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000
44,ifrs 17,account: investment component (how you identif...,derived from feed attributes,guaranteed indicator,guaranteed start date,guaranteed end date,death benefit,-1,-1,-1,-1,-1,-1,1,1,100.000000
45,ifrs 17,"account: premiums (first year, renewal, single)",derived from feed attributes,account,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,9,88.888889
46,ifrs 17,"account: premiums (first year, renewal, single)",derived from feed attributes,sub category code,benefit name,bill effective date,benefit effective date,-1,-1,-1,-1,-1,-1,1,9,11.111111
47,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,gross benefit,federal tax,provincial tax,deduction type,-1,-1,-1,-1,-1,-1,1,5,20.000000
48,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,pas system identifier,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,5,20.000000
49,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,pr key,line of business,-1,-1,-1,-1,-1,-1,-1,-1,2,5,40.000000
50,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,pr key,line of business,line of business,line of business,-1,-1,-1,-1,-1,-1,1,5,20.000000
51,ifrs 17,cohort / group,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,15,53.333333
52,ifrs 17,cohort / group,derived from feed attributes,major product,policy issue date,-1,-1,-1,-1,-1,-1,-1,-1,7,15,46.666667


In [116]:
# Sense check to ensure there are no duplicate attribute combinations for same derivation 

attribute_list = df_merge_col.iloc[:,0:13]
attribute_list = attribute_list[attribute_list['Reqcategory']=="ifrs 17"]

all_ifrs17 = attribute_list.FinancialReportingRequirements.unique()
attribute_list.drop("BUProposedStrategytoMeetRequirements", axis=1, inplace=True)
attribute_list.drop("Reqcategory", axis=1, inplace=True)

list_of_variation = attribute_list.values.tolist()
list_of_variation

duplicates = {}
counter = 0
for attrb in all_ifrs17:
    attribute_list_1 = attribute_list[attribute_list['FinancialReportingRequirements']==attrb]
    list_of_variation = attribute_list_1.values.tolist()
    for i in range(len(list_of_variation) - 1):
        set_of_dups = []
        if list_of_variation[i][0] not in duplicates:
            duplicates[list_of_variation[i][0]] = set_of_dups
        for j in range(i+1, len(list_of_variation)):
            if(set(list_of_variation[i]) == (set(list_of_variation[j]))):
                set_of_dups.append(str(list_of_variation[i][1:]))
                set_of_dups.append(str(list_of_variation[j][1:]))
                counter +=1
        if len(set_of_dups) > 0:
            duplicates[list_of_variation[i][0]].append(set_of_dups)
            
duplicates_1 = pd.DataFrame.from_dict(duplicates,orient='index').transpose()
duplicates_1
# duplicates_1.to_csv("duplicates_derived.csv")
# duplicates
# counter
# pd.DataFrame(duplicates.items(), columns=['Attribute Combination', 'Count'])
# attribute_list

,portfolio,"account: premiums (first year, renewal, single)",cohort / group,"cash flow indicator (cash / accrual / adjustment, etc.)"
0,None,None,None,"[['pr key', 'line of business', '-1', '-1', '-..."


In [97]:
duplicates_1.to_csv("duplicates_derived.csv")

In [117]:
df_merge_col['Derived Percent Max'] = df_merge_col.groupby(['FinancialReportingRequirements'])['Percentage'].transform(max)

df_merge_col

,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Attribute1_AttributeDefinition_ifnetnewdataattribute,Attribute2_AttributeDefinition_ifnetnewdataattribute,Attribute3_AttributeDefinition_ifnetnewdataattribute,Attribute4_AttributeDefinition_ifnetnewdataattribute,Attribute5_AttributeDefinition_ifnetnewdataattribute,Attribute6_AttributeDefinition_ifnetnewdataattribute,Attribute7_AttributeDefinition_ifnetnewdataattribute,Attribute8_AttributeDefinition_ifnetnewdataattribute,Attribute9_AttributeDefinition_ifnetnewdataattribute,Attribute10_AttributeDefinition_ifnetnewdataattribute,Count,Total_Derived,Percentage,Derived Percent Max
0,additional attribute,association of nonpar riders with the base plan,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,6,6,100.000000,100.000000
1,additional attribute,corporation,derived from feed attributes,issue,line of business,transaction type,account number,-1,-1,-1,-1,-1,-1,1,1,100.000000,100.000000
2,additional attribute,deduction amount,derived from feed attributes,deduction amount,deduction code,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000,100.000000
3,additional attribute,minor product,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,12,66.666667,66.666667
4,additional attribute,minor product,derived from feed attributes,minor product,-1,-1,-1,-1,-1,-1,-1,-1,-1,4,12,33.333333,66.666667
5,additional attribute,natural account,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,6,6,100.000000,100.000000
6,additional attribute,segment,derived from feed attributes,issue,line of business,transaction type,account number,case reimbursement code,-1,-1,-1,-1,-1,1,1,100.000000,100.000000
7,banking information,bank disbursement date,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,4,4,100.000000,100.000000
8,banking information,bank message,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,4,4,100.000000,100.000000
9,gl accounting requirements,account number,derived from feed attributes,amount,pdd indicator,ooc claims indicator / diff currency indicator,ooc claim amount / combined cheque amount,ooc currency code of payment by provider (jack...,-1,-1,-1,-1,-1,1,14,7.142857,57.142857


In [118]:
# Slice the derived by attribute only
sliced_derived = df_merge_col[["FinancialReportingRequirements", "BUProposedStrategytoMeetRequirements", "Derived Percent Max"]]
sliced_derived = sliced_derived.drop_duplicates()

In [119]:
# File Type: Derived Attribute Analysis

total_derived_file = summary_derived.groupby(['Typename', 'FinancialReportingRequirements', 'BUProposedStrategytoMeetRequirements']).size().reset_index(name='Total_Derived')
total_derived_f = total_derived_file[['Typename','FinancialReportingRequirements', 'Total_Derived']]
total_derived_f
duplicates_file = summary_derived.groupby(['Typename','Reqcategory', 'FinancialReportingRequirements', "BUProposedStrategytoMeetRequirements",'Attribute1_AttributeDefinition_ifnetnewdataattribute','Attribute2_AttributeDefinition_ifnetnewdataattribute','Attribute3_AttributeDefinition_ifnetnewdataattribute','Attribute4_AttributeDefinition_ifnetnewdataattribute','Attribute5_AttributeDefinition_ifnetnewdataattribute','Attribute6_AttributeDefinition_ifnetnewdataattribute','Attribute7_AttributeDefinition_ifnetnewdataattribute','Attribute8_AttributeDefinition_ifnetnewdataattribute','Attribute9_AttributeDefinition_ifnetnewdataattribute','Attribute10_AttributeDefinition_ifnetnewdataattribute']).size().reset_index(name='Count')
df_merge_col_file = pd.merge(duplicates_file, total_derived_f, on=['Typename','FinancialReportingRequirements'])
df_merge_col_file['Percentage'] = df_merge_col_file['Count'] / df_merge_col_file['Total_Derived'] * 100
df_merge_col_file

,Typename,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Attribute1_AttributeDefinition_ifnetnewdataattribute,Attribute2_AttributeDefinition_ifnetnewdataattribute,Attribute3_AttributeDefinition_ifnetnewdataattribute,Attribute4_AttributeDefinition_ifnetnewdataattribute,Attribute5_AttributeDefinition_ifnetnewdataattribute,Attribute6_AttributeDefinition_ifnetnewdataattribute,Attribute7_AttributeDefinition_ifnetnewdataattribute,Attribute8_AttributeDefinition_ifnetnewdataattribute,Attribute9_AttributeDefinition_ifnetnewdataattribute,Attribute10_AttributeDefinition_ifnetnewdataattribute,Count,Total_Derived,Percentage
0,ais,additional attribute,association of nonpar riders with the base plan,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000
1,ais,additional attribute,minor product,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000
2,ais,additional attribute,natural account,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000
3,ais,gl accounting requirements,account number,derived from feed attributes,base account,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000
4,ais,gl accounting requirements,debit / credit,derived from feed attributes,amount,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000
5,ais,gl accounting requirements,legal entity,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000
6,ais,gl accounting requirements,major product (segment/lob),derived from feed attributes,line of business,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000
7,ais,ifrs 17,"account: claims (e.g. death, annuity, etc.)",derived from feed attributes,claims,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000
8,ais,ifrs 17,"account: premiums (first year, renewal, single)",derived from feed attributes,account,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000
9,ais,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,pr key,line of business,-1,-1,-1,-1,-1,-1,-1,-1,2,3,66.666667


In [150]:
# File Type: Derived Attribute Analysis write to file 

df_merge_col_file['Derived Percent Max'] = df_merge_col_file.groupby(['FinancialReportingRequirements', "Typename"])['Percentage'].transform(max)
df_merge_col_file

,Typename,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Attribute1_AttributeDefinition_ifnetnewdataattribute,Attribute2_AttributeDefinition_ifnetnewdataattribute,Attribute3_AttributeDefinition_ifnetnewdataattribute,Attribute4_AttributeDefinition_ifnetnewdataattribute,Attribute5_AttributeDefinition_ifnetnewdataattribute,Attribute6_AttributeDefinition_ifnetnewdataattribute,Attribute7_AttributeDefinition_ifnetnewdataattribute,Attribute8_AttributeDefinition_ifnetnewdataattribute,Attribute9_AttributeDefinition_ifnetnewdataattribute,Attribute10_AttributeDefinition_ifnetnewdataattribute,Count,Total_Derived,Percentage,Derived Percent Max
0,ais,additional attribute,association of nonpar riders with the base plan,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000,100.000000
1,ais,additional attribute,minor product,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000,100.000000
2,ais,additional attribute,natural account,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000,100.000000
3,ais,gl accounting requirements,account number,derived from feed attributes,base account,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000,100.000000
4,ais,gl accounting requirements,debit / credit,derived from feed attributes,amount,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000,100.000000
5,ais,gl accounting requirements,legal entity,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000,100.000000
6,ais,gl accounting requirements,major product (segment/lob),derived from feed attributes,line of business,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000,100.000000
7,ais,ifrs 17,"account: claims (e.g. death, annuity, etc.)",derived from feed attributes,claims,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000,100.000000
8,ais,ifrs 17,"account: premiums (first year, renewal, single)",derived from feed attributes,account,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000,100.000000
9,ais,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,pr key,line of business,-1,-1,-1,-1,-1,-1,-1,-1,2,3,66.666667,66.666667


In [121]:
# File type: Slice the derived by attribute only

sliced_derived_file = df_merge_col_file[["Typename", "FinancialReportingRequirements", "BUProposedStrategytoMeetRequirements", "Derived Percent Max"]]
sliced_derived_file = sliced_derived_file.drop_duplicates()
sliced_derived_file

,Typename,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Derived Percent Max
0,ais,association of nonpar riders with the base plan,derived from feed attributes,100.000000
1,ais,minor product,derived from feed attributes,100.000000
2,ais,natural account,derived from feed attributes,100.000000
3,ais,account number,derived from feed attributes,100.000000
4,ais,debit / credit,derived from feed attributes,100.000000
5,ais,legal entity,derived from feed attributes,100.000000
6,ais,major product (segment/lob),derived from feed attributes,100.000000
7,ais,"account: claims (e.g. death, annuity, etc.)",derived from feed attributes,100.000000
8,ais,"account: premiums (first year, renewal, single)",derived from feed attributes,100.000000
9,ais,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,66.666667


In [122]:
strat_and_derived_file = pd.merge(summary_file, sliced_derived_file, on=["Typename","BUProposedStrategytoMeetRequirements","FinancialReportingRequirements"],how="outer")
strat_and_derived_file

,Reqcategory,Typename,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Feed Count,Percentage,Strat Percent Max,Derived Percent Max
0,additional attribute,ais,abc account number,attribute in feed,8.0,100,100.0,NaN
1,additional attribute,ais,ais - appfill,attribute in feed,8.0,100,100.0,NaN
2,additional attribute,ais,ais - descrfl,attribute in feed,8.0,100,100.0,NaN
3,additional attribute,ais,ais - match key,attribute in feed,8.0,100,100.0,NaN
4,additional attribute,ais,ais - refno,attribute in feed,8.0,100,100.0,NaN
5,additional attribute,ais,ais - rescent,attribute in feed,8.0,100,100.0,NaN
6,additional attribute,ais,ais - type code,attribute in feed,8.0,100,100.0,NaN
7,additional attribute,ais,association of nonpar riders with the base plan,derived from feed attributes,2.0,100,100.0,100.0
8,additional attribute,ais,base account,attribute in feed,8.0,100,100.0,NaN
9,additional attribute,ais,branch,attribute in feed,8.0,100,100.0,NaN


In [123]:
strat_and_derived = pd.merge(summary, sliced_derived, on=["BUProposedStrategytoMeetRequirements","FinancialReportingRequirements"],how="outer")
strat_and_derived

,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Feed Count,Percentage,Strat Percent Max,Derived Percent Max
0,additional attribute,1st billing pause,attribute in feed,1.0,100,100.000000,NaN
1,additional attribute,2nd billing pause,attribute in feed,1.0,100,100.000000,NaN
2,additional attribute,abc account number,attribute in feed,8.0,100,100.000000,NaN
3,additional attribute,account currency code,attribute in feed,1.0,100,100.000000,NaN
4,additional attribute,accounting - num of det line rec,attribute in feed,4.0,100,100.000000,NaN
5,additional attribute,accounting - num of transactions,attribute in feed,4.0,100,100.000000,NaN
6,additional attribute,accounting - total credit amount,attribute in feed,4.0,100,100.000000,NaN
7,additional attribute,accounting - total debit amount,attribute in feed,4.0,100,100.000000,NaN
8,additional attribute,accounting run date,attribute in feed,8.0,100,100.000000,NaN
9,additional attribute,accounting unit,attribute in feed,1.0,100,100.000000,NaN


In [124]:
# Data Definition Analysis

all_definitions = df[df['BUProposedStrategytoMeetRequirements']=='attribute in feed']
count_all_definitions = pd.DataFrame({'Total Feeds': all_definitions.groupby(['Reqcategory','FinancialReportingRequirements'])['FinancialReportingRequirements'].count()}).reset_index()

count_definitions = pd.DataFrame({'Feeds Using Datatype': all_definitions.groupby(['Reqcategory','FinancialReportingRequirements', 'BUProposedStrategytoMeetRequirements', 'DataType'])['DataType'].count()}).reset_index()
        
data_definitions = pd.merge(count_all_definitions, count_definitions, on=['Reqcategory',"FinancialReportingRequirements"],how="outer")

data_definitions['Percentage'] = data_definitions['Feeds Using Datatype'] / data_definitions['Total Feeds'] * 100
data_definitions

,Reqcategory,FinancialReportingRequirements,Total Feeds,BUProposedStrategytoMeetRequirements,DataType,Feeds Using Datatype,Percentage
0,additional attribute,1st billing pause,1,attribute in feed,varchar,1,100.000000
1,additional attribute,2nd billing pause,1,attribute in feed,varchar,1,100.000000
2,additional attribute,abc account number,8,attribute in feed,alphanumeric,8,100.000000
3,additional attribute,account currency code,1,attribute in feed,int,1,100.000000
4,additional attribute,accounting - num of det line rec,4,attribute in feed,character,4,100.000000
5,additional attribute,accounting - num of transactions,4,attribute in feed,character,4,100.000000
6,additional attribute,accounting - total credit amount,4,attribute in feed,character,4,100.000000
7,additional attribute,accounting - total debit amount,4,attribute in feed,character,4,100.000000
8,additional attribute,accounting run date,8,attribute in feed,character,8,100.000000
9,additional attribute,accounting unit,1,attribute in feed,alphanumeric,1,100.000000


In [125]:
data_definitions['Data Type Percent Max'] = data_definitions.groupby(['FinancialReportingRequirements'])['Percentage'].transform(max)

data_definitions_slice= data_definitions[["FinancialReportingRequirements", 'BUProposedStrategytoMeetRequirements', "Data Type Percent Max"]].drop_duplicates()
strat_derived_dtype = pd.merge(strat_and_derived, data_definitions_slice, on=["FinancialReportingRequirements", "BUProposedStrategytoMeetRequirements"],how="outer")
strat_derived_dtype.drop("Percentage", axis=1, inplace=True)

In [214]:
# Recommendations for derived

df_merge_col_file_rec = df_merge_col_file[(df_merge_col_file['Reqcategory']=='ifrs 17')]
df_merge_col_file_rec['Proposed_Attributes_To_Derive'] = df_merge_col_file_rec[['Attribute1_AttributeDefinition_ifnetnewdataattribute','Attribute2_AttributeDefinition_ifnetnewdataattribute','Attribute3_AttributeDefinition_ifnetnewdataattribute','Attribute4_AttributeDefinition_ifnetnewdataattribute','Attribute5_AttributeDefinition_ifnetnewdataattribute','Attribute6_AttributeDefinition_ifnetnewdataattribute','Attribute7_AttributeDefinition_ifnetnewdataattribute','Attribute8_AttributeDefinition_ifnetnewdataattribute','Attribute9_AttributeDefinition_ifnetnewdataattribute','Attribute10_AttributeDefinition_ifnetnewdataattribute']].apply(lambda x: ', '.join(x), axis=1)
df_merge_col_file_rec.drop(['Attribute1_AttributeDefinition_ifnetnewdataattribute','Attribute2_AttributeDefinition_ifnetnewdataattribute','Attribute3_AttributeDefinition_ifnetnewdataattribute','Attribute4_AttributeDefinition_ifnetnewdataattribute','Attribute5_AttributeDefinition_ifnetnewdataattribute','Attribute6_AttributeDefinition_ifnetnewdataattribute','Attribute7_AttributeDefinition_ifnetnewdataattribute','Attribute8_AttributeDefinition_ifnetnewdataattribute','Attribute9_AttributeDefinition_ifnetnewdataattribute','Attribute10_AttributeDefinition_ifnetnewdataattribute'], axis=1, inplace=True)
df_merge_col_file_rec = df_merge_col_file_rec[['Typename', 'Reqcategory','FinancialReportingRequirements','BUProposedStrategytoMeetRequirements',                      'Proposed_Attributes_To_Derive','Count',
 'Total_Derived','Percentage','Derived Percent Max']]
df_merge_col_file_rec['Proposed_Attributes_To_Derive'] = df_merge_col_file_rec.Proposed_Attributes_To_Derive.str.replace('-1' ,'')
df_merge_col_file_rec.rename(columns={'Derived Percent Max':'Derived_Percent_Max'}, inplace=True)
df_merge_col_file_rec['Majority_Attributes_To_Derive'] = df_merge_col_file_rec[df_merge_col_file_rec.Percentage.values == df_merge_col_file_rec.Derived_Percent_Max.values]["Proposed_Attributes_To_Derive"]
    
# test= df_merge_col_file_rec
# test["max_percentage"] = df_merge_col_file_rec.groupby(['FinancialReportingRequirements', "Typename"])['Percentage'].transform(max)
# test

# score = lambda x: x[x.Percentage.values == x.Derived_Percent_Max.values]["Proposed_Attributes_To_Derive"]
# df_merge_col_file_rec["test"] = df_merge_col_file_rec.groupby(['FinancialReportingRequirements', "Typename"]).apply(score)

df_merge_col_file_rec


,Typename,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Proposed_Attributes_To_Derive,Count,Total_Derived,Percentage,Derived_Percent_Max,Majority_Attributes_To_Derive
7,ais,ifrs 17,"account: claims (e.g. death, annuity, etc.)",derived from feed attributes,"claims, , , , , , , , ,",8,8,100.000000,100.000000,"claims, , , , , , , , ,"
8,ais,ifrs 17,"account: premiums (first year, renewal, single)",derived from feed attributes,"account, , , , , , , , ,",8,8,100.000000,100.000000,"account, , , , , , , , ,"
9,ais,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"pr key, line of business, , , , , , , ,",2,3,66.666667,66.666667,"pr key, line of business, , , , , , , ,"
10,ais,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"pr key, line of business, line of business, li...",1,3,33.333333,66.666667,NaN
11,ais,ifrs 17,cohort / group,derived from feed attributes,", , , , , , , , ,",8,8,100.000000,100.000000,", , , , , , , , ,"
12,ais,ifrs 17,date contract was issued (i.e. policy / benefi...,derived from feed attributes,", , , , , , , , ,",8,8,100.000000,100.000000,", , , , , , , , ,"
13,ais,ifrs 17,expense indicator to distinguish between maint...,derived from feed attributes,", , , , , , , , ,",8,8,100.000000,100.000000,", , , , , , , , ,"
14,ais,ifrs 17,pas system identifier,derived from feed attributes,"pr key, , , , , , , , ,",8,8,100.000000,100.000000,"pr key, , , , , , , , ,"
15,ais,ifrs 17,portfolio,derived from feed attributes,"major product, , , , , , , , ,",8,8,100.000000,100.000000,"major product, , , , , , , , ,"
19,direct,ifrs 17,cohort / group,derived from feed attributes,"major product, policy issue date, , , , , , , ,",3,3,100.000000,100.000000,"major product, policy issue date, , , , , , , ,"


In [213]:
df_merge_col_file_rec["Majority_Attributes_To_Derive"].fillna(df_merge_col_file_rec[df_merge_col_file_rec.Percentage.values == df_merge_col_file_rec.Derived_Percent_Max.values]["Proposed_Attributes_To_Derive"], inplace = True) 
df_merge_col_file_rec

,Typename,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Proposed_Attributes_To_Derive,Count,Total_Derived,Percentage,Derived_Percent_Max,Majority_Attributes_To_Derive
7,ais,ifrs 17,"account: claims (e.g. death, annuity, etc.)",derived from feed attributes,"claims, , , , , , , , ,",8,8,100.000000,100.000000,"claims, , , , , , , , ,"
8,ais,ifrs 17,"account: premiums (first year, renewal, single)",derived from feed attributes,"account, , , , , , , , ,",8,8,100.000000,100.000000,"account, , , , , , , , ,"
9,ais,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"pr key, line of business, , , , , , , ,",2,3,66.666667,66.666667,"pr key, line of business, , , , , , , ,"
10,ais,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"pr key, line of business, line of business, li...",1,3,33.333333,66.666667,test
11,ais,ifrs 17,cohort / group,derived from feed attributes,", , , , , , , , ,",8,8,100.000000,100.000000,", , , , , , , , ,"
12,ais,ifrs 17,date contract was issued (i.e. policy / benefi...,derived from feed attributes,", , , , , , , , ,",8,8,100.000000,100.000000,", , , , , , , , ,"
13,ais,ifrs 17,expense indicator to distinguish between maint...,derived from feed attributes,", , , , , , , , ,",8,8,100.000000,100.000000,", , , , , , , , ,"
14,ais,ifrs 17,pas system identifier,derived from feed attributes,"pr key, , , , , , , , ,",8,8,100.000000,100.000000,"pr key, , , , , , , , ,"
15,ais,ifrs 17,portfolio,derived from feed attributes,"major product, , , , , , , , ,",8,8,100.000000,100.000000,"major product, , , , , , , , ,"
19,direct,ifrs 17,cohort / group,derived from feed attributes,"major product, policy issue date, , , , , , , ,",3,3,100.000000,100.000000,"major product, policy issue date, , , , , , , ,"


In [159]:
summary_file_rec = summary_file[(summary_file['Reqcategory']=='ifrs 17')]

summary_file_rec['Majority_Strategy'] = summary_file_rec.groupby(['FinancialReportingRequirements', "Typename"])['BUProposedStrategytoMeetRequirements'].transform(max)
summary_file_rec

,Reqcategory,Typename,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Feed Count,Percentage,Strat Percent Max,Majority_Strategy
504,ifrs 17,ais,"account: claims (e.g. death, annuity, etc.)",derived from feed attributes,8.0,100,100.0,derived from feed attributes
505,ifrs 17,ais,account: directly attributable acquisition costs,attribute not required,8.0,100,100.0,attribute not required
506,ifrs 17,ais,account: investment component (how you identif...,undecided,8.0,100,100.0,undecided
507,ifrs 17,ais,"account: premiums (first year, renewal, single)",derived from feed attributes,8.0,100,100.0,derived from feed attributes
508,ifrs 17,ais,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,3.0,37.5,62.5,undecided
509,ifrs 17,ais,cash flow indicator (cash / accrual / adjustme...,undecided,5.0,62.5,62.5,undecided
510,ifrs 17,ais,cohort / group,derived from feed attributes,8.0,100,100.0,derived from feed attributes
511,ifrs 17,ais,conversions: indicator that policy is result o...,undecided,8.0,100,100.0,undecided
512,ifrs 17,ais,conversions: old policy or contract number and...,undecided,8.0,100,100.0,undecided
513,ifrs 17,ais,date contract was issued (i.e. policy / benefi...,derived from feed attributes,8.0,100,100.0,derived from feed attributes


In [76]:
file_types_all.to_excel(writer, sheet_name='File Types_by BU')
# Overall analysis by attribute
bu_strategy.to_excel(writer, sheet_name='IFRS 17 Decided_By BU')
# Break down of analysis by attribute
strat_derived_dtype.to_excel(writer, sheet_name='Detailed Analysis_by Attrb')
df_merge_col.to_excel(writer, sheet_name='Strat for Derived_by Attrb')
data_definitions.to_excel(writer, sheet_name='Data Type Breakdown_by Attrb')
# Break down of analysis by file type
strat_and_derived_file.to_excel(writer, sheet_name='Detailed Analysis_by File Type')
df_merge_col_file.to_excel(writer, sheet_name='Strat for Derived_by File Type')
